In [4]:
import pandas as pd
import os
from pgtools import gff_parser, maf_parser

In [8]:
data_dir = "/home/pampuch/studia/magisterka/test_data/klebsiella_subset_old/"
maf_dir = os.path.join(data_dir, "cactus.maf")
gffs_dir = os.path.join(data_dir, "gff")

### Using bedttools substract output

1. Substract full_scaffolds and model you want to check coverage for
2. cvg: sum of lens in full contigs gff - sum of lens in substract out / sum of lens in full contigs gff

In [1]:
gff_dir = "/home/pampuch/studia/magisterka/final_summaries/substract.gff"

In [2]:
def get_cds_sum_aggregate_lens(gff):
    # cds_lens = {}
    cds_lens = 0
    gff = gff_parser.parse_joined_gff(gff)
    for cds in gff:
        cds_len = cds.end - cds.start + 1
        cds_lens += cds_len
        # if cds.seq_name in cds_lens:
        #     if cds.annotation_id in cds_lens[cds.seq_name]:
        #         cds_lens[cds.seq_name][cds.annotation_id] += cds_len
        #     else:
        #         cds_lens[cds.seq_name][cds.annotation_id] = cds_len
        # else:
        #     cds_lens[cds.seq_name] = {cds.annotation_id : cds_len}
    return cds_lens

In [5]:
sum_not_present_lens = get_cds_sum_aggregate_lens(gff_dir)

In [6]:
sum_not_present_lens

3234700

In [12]:
inter_gff = "scaff_inter_model.gff"
full_gff = "full.gff"
gffs_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/model_cvg"
genome_full_lens = {}
for dataset in os.listdir(os.path.join(gffs_dir, "full_scaffolds")):
    genome_full_lens[dataset] = get_cds_sum_aggregate_lens(os.path.join(gffs_dir, "full_scaffolds", dataset, full_gff))

In [13]:
genome_full_lens["GPSC_subset"]

45647946

In [16]:
(genome_full_lens["GPSC_subset"] - sum_not_present_lens)/ genome_full_lens["GPSC_subset"]

0.929138104045251

In [23]:
cactus_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/full_sub_sub_cactus"
full_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/full_scaffolds/"

In [24]:
dataset = "klebsiella_subset"
model_gff = os.path.join(cactus_dir, dataset, "model_clean.gff")
full_gff = os.path.join(full_dir, dataset, "full.gff")

get_cds_sum_aggregate_lens(model_gff) / get_cds_sum_aggregate_lens(full_gff)

0.21505028592800488

In [42]:
clean_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/clean_model_gffs/"
inter_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/models_intersections/"
dataset = "klebsiella_subset"
model_gff = os.path.join(clean_dir, "panaroo", dataset, "clean.gff")
full_gff = os.path.join(full_dir, dataset, "full.gff")

get_cds_sum_aggregate_lens(model_gff) / get_cds_sum_aggregate_lens(full_gff)

0.808933061171146

In [28]:
for dir in os.listdir(os.path.join(full_dir)):
    print(dir)

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
klebsiella_subset
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
GPSC_subset


In [45]:
def datasets_mapping(x):
    if x.startswith("sim"):
        x= x.replace("sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1", "E. coli ")
    else:
        x= x.replace("_subset", "")
        x= x.replace("GPSC", "Pneumococcus")
        x = x.replace("klebsiella", "Klebsiella")
    return x.strip().replace("_","")

In [32]:
def model_map(x):
    map_dict = {"panaroo":"Panaroo", "progressive" : "Progressive-C.", "cactus": "Minigraph-C."}
    return map_dict[x]
model_map("panaroo")

'Panaroo'

In [48]:
columns = ["Dataset", "Liczba pozycji w danych wejściowych", "Liczba pozycji w modelu Panaroo", "Liczba pozycji w modelu Minigraph-C.", "Liczba pozycji w modelu Progressive-C.", "Liczba pozycji występujących zarówno w Panaroo i Minigraph-C.","Liczba pozycji występujących zarówno w Panaroo i Progressive-C."]

In [44]:
res_dict = {}
for dataset in os.listdir(os.path.join(full_dir)):
    res_dict[dataset] = []
    res_dict[dataset].append(get_cds_sum_aggregate_lens(os.path.join(full_dir, dataset, "full.gff")))
    for model in ["panaroo", "cactus", "progressive"]:
        print(model)
        if model == "progressive" and (dataset.endswith("rep1") or dataset.endswith("contaminated")):
            res_dict[dataset].append("X")
        else:
            res_dict[dataset].append(get_cds_sum_aggregate_lens(os.path.join(clean_dir, model, dataset, "clean.gff")))
    res_dict[dataset].append(get_cds_sum_aggregate_lens(os.path.join(inter_dir, "cactus_inter_panaroo", dataset, "inter.gff")))
    if model == "progressive" and (dataset.endswith("rep1") or dataset.endswith("contaminated")):
        res_dict[dataset].append("X")
    else:
        res_dict[dataset].append(get_cds_sum_aggregate_lens(os.path.join(inter_dir, "progressive_inter_panaroo", dataset, "inter.gff")))

res_dict


panaroo
cactus
progressive
panaroo
cactus
progressive
panaroo
cactus
progressive
panaroo
cactus
progressive
panaroo
cactus
progressive


{'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': [347041411,
  281922719,
  143492759,
  333856098,
  115536907,
  274203513],
 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': [348677091,
  290618642,
  320243952,
  'X',
  269177229,
  'X'],
 'klebsiella_subset': [117114971,
  94738172,
  25185608,
  96114588,
  20343729,
  82450108],
 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1': [346957477,
  290782037,
  323942286,
  'X',
  272634861,
  'X'],
 'GPSC_subset': [45647946, 38159317, 16438469, 38725326, 13854782, 33156674]}

In [46]:
rows = []
for k, v in res_dict.items():
    rows.append([datasets_mapping(k)] + v)

In [49]:
df = pd.DataFrame(columns = columns, data = rows)

In [51]:
df = df.sort_values(by="Dataset")

In [52]:
df

,Dataset,Liczba pozycji w danych wejściowych,Liczba pozycji w modelu Panaroo,Liczba pozycji w modelu Minigraph-C.,Liczba pozycji w modelu Progressive-C.,Liczba pozycji występujących zarówno w Panaroo i Minigraph-C.,Liczba pozycji występujących zarówno w Panaroo i Progressive-C.
3,E. coli,346957477,290782037,323942286,X,272634861,X
1,E. coli contaminated,348677091,290618642,320243952,X,269177229,X
0,E. coli fragmented,347041411,281922719,143492759,333856098,115536907,274203513
2,Klebsiella,117114971,94738172,25185608,96114588,20343729,82450108
4,Pneumococcus,45647946,38159317,16438469,38725326,13854782,33156674


### Modify
1. Add panaroo joined
2. Dataset, 

In [53]:
df.to_csv("/home/pampuch/studia/magisterka/final_summaries/pozycje_wspólne.csv", index=False)

In [34]:
for model in os.listdir(inter_dir):
    print(model)

maf_comp_trimmed
core_stats.csv
boxplots
annots_summaries
gfa_vs_maf_res_new
block_stats_hists_new
klebsiella_subset_trp
panaroo_compl
struct_triplets
com_basic.csv
klebsiella_subset_trp.tar.gz
gfa_vs_maf_summary.csv
stats.csv
bedtools_core
scaf_cover
core_summaries_old
core_summaries_new
annots_summaries.tar.gz
basic_dataset_stats.csv
gffs
substract.gff
genomes_cvg.csv
blocks_basic_stats_new
core_new.csv
scaffols_lens
com_trick.csv
gffs_old


In [4]:
# gffs = gff_parser.parse_GFFs_dir(gffs_dir, gff_simple=False)

In [5]:
# def all_scaffs_to_gff(gffs, out_file):
#     res_gff = open(out_file, "w")
#     # gff_all = gff_parser.parse_GFFs_dir(gffs, gff_simple=False)
#     for genome, gff in gffs.items():
#         for scaff in gff.scaffolds:
#             # cont_n += 1
#             # cont_lens.append(scaff.length)
#             seq_name = f"{scaff.genome}.{scaff.name}"
#             start = 1
#             end = scaff.length
#             strand_sign = "+"
#             res_gff.write(f"{seq_name}\tunidentified\tunidentified\t{start}\t{end}\t.\t{strand_sign}\t0\tINFO=whole_scaffold\n")


In [6]:
# all_scaffs_to_gff(gffs, "/home/pampuch/studia/magisterka/test_data/scaff.gff")

In [8]:
inter_gff = "scaff_inter_model.gff"
full_gff = "full.gff"
gffs_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/model_cvg"

In [9]:
for dir in os.listdir(gffs_dir):
    print(dir)

panaroo_pangenome_cvg
panaroo_joined_pangenome_cvg
full_scaffolds
panaroo_compl_pangenome_cvg
cact_pangenome_cvg
prog_pangenome_cvg


In [10]:
def datasets_mapping(x):
    if x.startswith("sim"):
        x= x.replace("sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1", "E. coli ")
    else:
        x= x.replace("_subset", "")
        x= x.replace("GPSC", "pneumococcus")
    return x.strip().replace("_","")

In [11]:
for dir in os.listdir(gffs_dir):
    if not dir.endswith("cvg"):
        continue
    print(dir)

panaroo_pangenome_cvg
panaroo_joined_pangenome_cvg
panaroo_compl_pangenome_cvg
cact_pangenome_cvg
prog_pangenome_cvg


In [12]:
genome_full_lens = {}
for dataset in os.listdir(os.path.join(gffs_dir, "full_scaffolds")):
    genome_full_lens[dataset] = get_cds_sum_aggregate_lens(os.path.join(gffs_dir, "full_scaffolds", dataset, full_gff))

In [13]:
genome_full_lens

{'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': 347041411,
 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': 348677091,
 'klebsiella_subset': 117114971,
 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1': 346957477,
 'GPSC_subset': 45647946}

In [48]:
models_full_lens = {}
for model in os.listdir(gffs_dir):
    if not model.endswith("cvg"):
        continue
    models_full_lens[model] = {}
    for dataset in os.listdir(os.path.join(gffs_dir, model)):
        file = os.path.join(gffs_dir, model, dataset, inter_gff)
        models_full_lens[model][dataset] = get_cds_sum_aggregate_lens(file)

In [45]:
def model_mapping(model):
    name_dict = {
    "panaroo_pangenome_cvg" : "Panaroo podstawowy",
    "panaroo_joined_pangenome_cvg" : "Panaroo z uzupełnieniem",
    "panaroo_compl_pangenome_cvg" : "Uzupełnienie Panaroo",
    "cact_pangenome_cvg" : "Minigraph-C",
    "prog_pangenome_cvg" : "Progressive-C."
    }
    # model_name = name_dict(model)
    return name_dict[model]

models_order = ["Panaroo podstawowy", "Uzupełnienie Panaroo", "Panaroo z uzupełnieniem", "Minigraph-C", "Progressive-C."]

In [31]:
models_full_lens

{'panaroo_pangenome_cvg': {'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': 282541528,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': 291247487,
  'klebsiella_subset': 94901769,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1': 291403166,
  'GPSC_subset': 38248115},
 'panaroo_joined_pangenome_cvg': {'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': 309664959,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': 322460315,
  'klebsiella_subset': 103430107,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1': 323067275,
  'GPSC_subset': 49742151},
 'panaroo_compl_pangenome_cvg': {'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': 29577675,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': 31344449,
  'klebsiella_subset': 8746124,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1': 31723693,
  'GPSC_subset': 11230906},
 'cact_pangenome_cvg': {'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': 143537727,
  'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': 320273843,
  'klebsiella_subset': 25184748,
  'sim_gr

In [49]:
models_cvg = {}

In [50]:
for model, datasets in models_full_lens.items():
    # model_name = model_mapping(model)
    models_cvg[model_mapping(model)] = {}
    for dataset, length in datasets.items():
        models_cvg[model_mapping(model)].update({datasets_mapping(dataset) : length / genome_full_lens[dataset]})

In [51]:
models_cvg

{'Panaroo podstawowy': {'E. coli fragmented': 0.8141435547586567,
  'E. coli contaminated': 0.8352928670039867,
  'klebsiella': 0.8103299534608602,
  'E. coli': 0.8398814993688694,
  'pneumococcus': 0.83789345089043},
 'Panaroo z uzupełnieniem': {'E. coli fragmented': 0.8922997348002368,
  'E. coli contaminated': 0.924810729822224,
  'klebsiella': 0.8831501738577897,
  'E. coli': 0.9311437176492957,
  'pneumococcus': 1.0896908921159345},
 'Uzupełnienie Panaroo': {'E. coli fragmented': 0.08522808535953076,
  'E. coli contaminated': 0.08989534962020203,
  'klebsiella': 0.07467981185770008,
  'E. coli': 0.09143395114093478,
  'pneumococcus': 0.24603310738231246},
 'Minigraph-C': {'E. coli fragmented': 0.41360403240177007,
  'E. coli contaminated': 0.9185399651048483,
  'klebsiella': 0.21504294271652086,
  'E. coli': 0.93384019506229,
  'pneumococcus': 0.3600499352150478},
 'Progressive-C.': {'E. coli fragmented': 0.9634378273087415,
  'klebsiella': 0.835400249554773,
  'pneumococcus': 0.8

In [36]:
models_cvg.keys()

dict_keys(['panaroo_pangenome_cvg', 'panaroo_joined_pangenome_cvg', 'panaroo_compl_pangenome_cvg', 'cact_pangenome_cvg', 'prog_pangenome_cvg'])

In [ ]:
def model_mapping(model):
    name_dict = {
    "panaroo_pangenome_cvg" : "Panaroo podstawowy",
    "panaroo_joined_pangenome_cvg" : "Panaroo z uzupełnieniem",
    "panaroo_compl_pangenome_cvg" : "Uzupełnienie Panaroo",
    "cact_pangenome_cvg" : "Minigraph-C",
    "prog_pangenome_cvg" : "Progressive-C."
    }
    return name_dict(model)

models_order = ["Panaroo podstawowy", "Uzupełnienie Panaroo", "Panaroo z uzupełnieniem", "Minigraph-C", "Progressive-C."]

In [52]:
datasets = list(models_cvg["Panaroo podstawowy"].keys())

In [57]:
datasets = sorted(datasets)

In [35]:
columns = ["Dataset", "Model", "Pokrycie pangenomu"]

In [60]:
rows = []
for dataset in datasets:
    for model in models_order:
        try:
            rows.append([dataset, model, models_cvg[model][dataset]])
        except:
            continue

In [64]:
df = pd.DataFrame(data=rows, columns=columns)
df["Pokrycie pangenomu"] = df["Pokrycie pangenomu"]*100

In [65]:
df

,Dataset,Model,Pokrycie pangenomu
0,E. coli,Panaroo podstawowy,83.988150
1,E. coli,Uzupełnienie Panaroo,9.143395
2,E. coli,Panaroo z uzupełnieniem,93.114372
3,E. coli,Minigraph-C,93.384020
4,E. coli contaminated,Panaroo podstawowy,83.529287
5,E. coli contaminated,Uzupełnienie Panaroo,8.989535
6,E. coli contaminated,Panaroo z uzupełnieniem,92.481073
7,E. coli contaminated,Minigraph-C,91.853997
8,E. coli fragmented,Panaroo podstawowy,81.414355
9,E. coli fragmented,Uzupełnienie Panaroo,8.522809


In [63]:
df.to_csv("/home/pampuch/studia/magisterka/final_summaries/genomes_cvg.csv")